In [99]:
import os
import numpy as np
import torch
from torch_geometric.utils import dense_to_sparse
from six.moves import urllib
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pandas as pd
import pickle
import torch
from tqdm import tqdm
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric_temporal.nn.recurrent import A3TGCN2
from torch_geometric_temporal.signal import temporal_signal_split
from torch.optim.lr_scheduler import ExponentialLR
Project_Path='/Users/nickkarras/PycharmProjects/ParkingViolationPredictionGraph_Git'

<h1>
<center>A3T-GCN: Attention Temporal Graph
Convolutional Network for Parking Violataion Forecasting</center>
</h1>


In [103]:
def paths(Project_Path):
    TestDataset=Project_Path+ '/Data/Test_Dataset_Graph.pkl'
    TestTargets=Project_Path+ '/Data/Test_Rate_Timeseries.csv'
    TestMask=Project_Path+ '/Data/Test_Mask.csv'
    TrainDataset =Project_Path+ '/Data/Dataset_Graph.pkl'
    TrainTargets=Project_Path+ '/Data/STS_Rate_Timeseries.csv'

    return TestDataset,TestTargets,TestMask,TrainDataset,TrainTargets

# EDGES & EDGE WEIGHTS

In [108]:
def edge_weights(Project_Path):
    path = Project_Path+'/Data/EdgesDuration250.csv'
    weights = 'Duration'
    Edge_List=pd.read_csv(path,sep=',', index_col=0)
    Distance=np.array(Edge_List[weights])
    print(Distance.shape)
    Edge_List=Edge_List.drop([weights], axis=1)
    Edges=np.array(Edge_List)
    Edges=Edges.T
    print(Edges.shape)
    return Edges,Distance

In [109]:
TestDataset,TestTargets,TestMask,TrainDataset,TrainTargets=paths(Project_Path)
Edges,Distance=edge_weights(Project_Path)

(3030,)
(2, 3030)


# GRAPH NO TEMPORAL

In [75]:
with open(TrainDataset, 'rb') as inp:
    Train_Dataset = pickle.load(inp)
Name=['Date_Sin','Holidays','Capacity','temp','humidity','Week_Day_Sin','Month_Sin','Real_Time','Γενικό Νοσοκομείο Θεσσαλονίκης «Γ. Γεννηματάς»', 'Λιμάνι' ,'Δημαρχείο Θεσσαλονίκης','Λευκός Πύργος','Αγορά Καπάνι','Λαδάδικα','Πλατεία Άθωνος','Πλατεία Αριστοτέλους','Ροτόντα','Πλατεία Αγίας Σοφίας','Πλατεία Αντιγονιδών','Μουσείο Μακεδονικού Αγώνα','Πλατεία Ναυαρίνου','Πάρκο ΧΑΝΘ','Ιερός Ναός Αγίου Δημητρίου','ΔΕΘ','ΑΠΘ','Άγαλμα Ελευθερίου Βενιζέλου','Ρωμαϊκή Αγορά Θεσσαλονίκης','Predictions']
for i in range (0,len(Train_Dataset)):
    Train_Dataset[i]=Train_Dataset[i].sort_values("Slot_id")
    Train_Dataset[i]=Train_Dataset[i].reset_index()
    Train_Dataset[i]=Train_Dataset[i].drop(['index'], axis=1)
    Train_Dataset[i]=Train_Dataset[i][Name]


Rate_Timeseries=np.array(Train_Dataset)
Rate_Timeseries=np.reshape(Rate_Timeseries, (len(Train_Dataset), 222,28,1))
print(Rate_Timeseries.shape)

Target=pd.read_csv(TrainTargets,sep=',', index_col=0)
Target=np.array(Target)
Target=Target.T
Target=np.reshape(Target, (len(Train_Dataset), 222,1))
print(Target.shape)

graTrain=StaticGraphTemporalSignal(edge_index=Edges,edge_weight=Distance,features=Rate_Timeseries,targets=Target)
print("Dataset type:  ", graTrain)
print(next(iter(graTrain)))

(5074, 222, 28, 1)
(5074, 222, 1)
Dataset type:   <torch_geometric_temporal.signal.static_graph_temporal_signal.StaticGraphTemporalSignal object at 0x17be92460>
Data(x=[222, 28, 1], edge_index=[2, 3030], edge_attr=[3030], y=[222, 1])


In [111]:
with open(TestDataset, 'rb') as inp:
    Test_Dataset = pickle.load(inp)
Name=['Date_Sin','Holidays','Capacity','temp','humidity','Week_Day_Sin','Month_Sin','Real_Time','Γενικό Νοσοκομείο Θεσσαλονίκης «Γ. Γεννηματάς»', 'Λιμάνι' ,'Δημαρχείο Θεσσαλονίκης','Λευκός Πύργος','Αγορά Καπάνι','Λαδάδικα','Πλατεία Άθωνος','Πλατεία Αριστοτέλους','Ροτόντα','Πλατεία Αγίας Σοφίας','Πλατεία Αντιγονιδών','Μουσείο Μακεδονικού Αγώνα','Πλατεία Ναυαρίνου','Πάρκο ΧΑΝΘ','Ιερός Ναός Αγίου Δημητρίου','ΔΕΘ','ΑΠΘ','Άγαλμα Ελευθερίου Βενιζέλου','Ρωμαϊκή Αγορά Θεσσαλονίκης','Predictions']
for i in range (0,len(Test_Dataset)):
    Test_Dataset[i]=Test_Dataset[i].sort_values("Slot_id")
    Test_Dataset[i]=Test_Dataset[i].reset_index()
    Test_Dataset[i]=Test_Dataset[i].drop(['index'], axis=1)
    Test_Dataset[i]=Test_Dataset[i][Name]

Rate_Timeseries=np.array(Test_Dataset)
Rate_Timeseries=np.reshape(Rate_Timeseries, (len(Test_Dataset), 222,28,1))
print(Rate_Timeseries.shape)

Target=pd.read_csv(TestTargets,sep=',', index_col=0)

Target=np.array(Target)
Target=Target.T
Target=np.reshape(Target, (len(Test_Dataset), 222,1))
print(Target.shape)

graTest=StaticGraphTemporalSignal(edge_index=Edges,edge_weight=Distance,features=Rate_Timeseries,targets=Target)
print("Dataset type:  ", graTest)
print(next(iter(graTest)))

(609, 222, 28, 1)
(609, 222, 1)
Dataset type:   <torch_geometric_temporal.signal.static_graph_temporal_signal.StaticGraphTemporalSignal object at 0x17d9cd8b0>
Data(x=[222, 28, 1], edge_index=[2, 3030], edge_attr=[3030], y=[222, 1])


# Graph Temporal

# Test Train Split

In [112]:
train_dataset=graTrain
test_dataset=graTest

print("Number of train buckets: ", len(set(train_dataset)))
print("Number of test buckets: ", len(set(test_dataset)))

Number of train buckets:  5074
Number of test buckets:  609


# Creating DataLoaders


In [113]:
batch_size=64

In [114]:
train_input = np.array(train_dataset.features) # (5074, 222, 24, 1)
train_target = np.array(train_dataset.targets) # (5074, 207, 1)
train_x_tensor = torch.from_numpy(train_input).type(torch.FloatTensor)  # (B, N, F, T)
train_target_tensor = torch.from_numpy(train_target).type(torch.FloatTensor)  # (B, N, T)
train_dataset_new = torch.utils.data.TensorDataset(train_x_tensor, train_target_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset_new, batch_size=batch_size,drop_last=True)

In [115]:
test_input = np.array(test_dataset.features) # (, 207, 2, 12)
test_target = np.array(test_dataset.targets) # (, 207, 12)
test_x_tensor = torch.from_numpy(test_input).type(torch.FloatTensor)# (B, N, F, T)
test_target_tensor = torch.from_numpy(test_target).type(torch.FloatTensor) # (B, N, T)
test_dataset_new = torch.utils.data.TensorDataset(test_x_tensor, test_target_tensor)
test_loader = torch.utils.data.DataLoader(test_dataset_new, batch_size=batch_size,drop_last=True)

## Model

Which model to choose depends on which time-series task you work on. 

- A3TGCN is an extension of TGCN that uses attention 
- The spatial aggregation uses GCN, the temporal aggregation a GRU
- We can pass in periods to get an embedding for several timesteps
- This embedding can be used to predict several steps into the future = output dimension
- We could also do this in a loop and feed it again into the model (would be autoregressive)
- There is only one block here. Other layers also allow stacking???

<html>
<img src="https://i.ibb.co/WxrJQbc/a3tgcn.png", height="300"></img>

# TGCN model
A temporal GCN (T-GCN) model was constructed by combining GCN and GRU. 

n historical time series traffic data were inputted into the T-GCN model to obtain n hidden states (h) that covered spatiotemporal characteristics:{h(t−n), · · · , h(t−1), h(t)}


ut = σ(Wu ∗ (GC(A, Xt), ht−1)) 

rt = σ(Wr ∗ (GC(A, Xt), ht−1)) 

ct = tanh(Wc ∗ (GC(A, Xt), (rt ∗ ht−1)))

ht = ut ∗ ht−1 + (1 − ut) ∗ ct) 


Then, the hidden states were inputted into the attention model to determine the context vector that covers the global traffic variation information. Particularly, the weight of each h was calculated by Softmax using a multilayer perception:{at−n, · · · , at−1, at}.The context vector that covers global traffic variation information is calculated by the weighted sum. 


# A3TGCN Model
The A3TGCN is an extention of the TGCN model by adding an attention mechanism.

The attention mechanism was introduced to re-weight the influence of historical traffic states and thus to capture the global variation trends of traffic state

In [116]:
from torch.nn import Linear
from torch.nn import ReLU
import torch.nn as nn
from torch.nn.init import kaiming_uniform_
from tqdm import tqdm

In [117]:
class TemporalGNN(torch.nn.Module):
    def __init__(self, node_features, periods, batch_size):
        super(TemporalGNN, self).__init__()
        # Attention Temporal Graph Convolutional Cell
        
        self.tgnn = A3TGCN2(in_channels=node_features,  out_channels=128, periods=periods,batch_size=batch_size) # node_features=2, periods=12
        
        
        self.hidden2 = Linear(128, 64)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        
        self.lstm = nn.LSTM(64, 32, 1, batch_first=True)

        
        self.hidden4 = Linear(32, 16)
        kaiming_uniform_(self.hidden4.weight, nonlinearity='relu')
        self.act4 = ReLU()

        self.linear = torch.nn.Linear(16, periods)
        

    def forward(self, x, edge_index):
        h = self.tgnn(x, edge_index) # x [b, 222, 28, 1]  returns h [b, 222, 1]
        h = F.relu(h) 
        
        h0 = torch.zeros(1, h.size(0), 32).requires_grad_()
        c0 = torch.zeros(1, h.size(0), 32).requires_grad_()
        
        h = self.hidden2(h)
        h = self.act2(h)

        out, (hn, cn) = self.lstm(h, (h0.detach(), c0.detach()))

        out = self.hidden4(out)
        out = self.act4(out)

        out = self.linear(out) 

        return out

TemporalGNN(node_features=28, periods=1, batch_size=batch_size)

TemporalGNN(
  (tgnn): A3TGCN2(
    (_base_tgcn): TGCN2(
      (conv_z): GCNConv(28, 128)
      (linear_z): Linear(in_features=256, out_features=128, bias=True)
      (conv_r): GCNConv(28, 128)
      (linear_r): Linear(in_features=256, out_features=128, bias=True)
      (conv_h): GCNConv(28, 128)
      (linear_h): Linear(in_features=256, out_features=128, bias=True)
    )
  )
  (hidden2): Linear(in_features=128, out_features=64, bias=True)
  (act2): ReLU()
  (lstm): LSTM(64, 32, batch_first=True)
  (hidden4): Linear(in_features=32, out_features=16, bias=True)
  (act4): ReLU()
  (linear): Linear(in_features=16, out_features=1, bias=True)
)

## Training

# Loading the graph once 
because it's a static graph

In [118]:
for snapshot in train_dataset:
    static_edge_index = snapshot.edge_index
    break;

In [119]:
def GetTest_MAE_MSE(model,test_loader,metric_fn,TestMask):
    model.eval()
    step = 0
    total_loss = []
    Predictions=[]
    true=[]
    Test_Mask=pd.read_csv(TestMask,index_col=0)
#     cols=[0]
#     Test_Mask = Test_Mask.drop(Test_Mask.columns[cols],axis=1)
    Test_Mask['New']=0
    Test_Mask=Test_Mask.T
    Test_Mask=Test_Mask.values.tolist()
    P=[]
    R=[]
    for encoder_inputs, labels in test_loader:
        # Get model predictions
        y_hat = model(encoder_inputs, static_edge_index)
        # Mean squared error
        for ii in range (0,len(y_hat)):
            Predictions.append(y_hat[ii])

        for kk in range (0,len(labels)):
            true.append(labels[kk])


        loss = metric_fn(y_hat, labels)
        total_loss.append(loss.item())
    
    for i in range(0,(len(Predictions))):
        for k in range (0,len(Test_Mask[0])):
            if Test_Mask[i][k]==1:
                P1=Predictions[i][k]
                P.append(float(P1))
                R1=true[i][k]
                R.append(float(R1))
    MAE=tf.keras.metrics.mean_absolute_error(R, P)
    MSE=tf.keras.metrics.mean_squared_error(R, P)
    return MAE,MSE

In [ ]:
model = TemporalGNN(node_features=28, periods=1, batch_size=batch_size)
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()
metric_fn=torch.nn.L1Loss()

model.train()
Test_MAE=[]
Test_MSE=[]
Train_Loss=[]
for epoch in range(1,61):
    step = 0
    loss_list = []
    for encoder_inputs, labels in tqdm(train_loader):
        y_hat = model(encoder_inputs, static_edge_index)         # Get model predictions
        loss = loss_fn(y_hat, labels) # Mean squared error #loss = torch.mean((y_hat-labels)**2)  sqrt to change it to rmse
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        step= step+ 1
        loss_list.append(loss.item())
    
    Loss=sum(loss_list)/len(loss_list)
    print("Epoch {} train MSE: {:.4f}".format(epoch, Loss))
    MAE,MSE=GetTest_MAE_MSE(model,test_loader,metric_fn,TestMask)
    Test_MAE.append(float(MAE))
    Test_MSE.append(float(MSE))
    Train_Loss.append(Loss)
    print("Test MAE: {:.4f}".format(MAE))
    print("Test MSE: {:.4f}".format(MSE))

 87%|████████▋ | 69/79 [00:31<00:04,  2.18it/s]